In [100]:
import os
import json
from opensearchpy import OpenSearch, RequestsHttpConnection, helpers
import boto3
from requests_aws4auth import AWS4Auth
from openai import OpenAI
from config import *

In [58]:
# ─── Configuration ──────────────────────────────────────────────────────────────
DOMAIN = "search-redcoat-express-wvba3hxtx72luhtzgh7tb56k4i.aos.eu-north-1.on.aws"
REGION = "eu-north-1"
INDEX  = "redcoat-express"
MASTER_USER        = "admin"              # the internal username you configured
MASTER_PASSWORD    = "ASINnumber1!" # the password you set

DATA_DIRS = [email_chunks_dir, attachment_chunks_dir, thread_documents_dir]

Access over IAM (Switched Off)

In [59]:
# # Creating a session for a specific local user "rag-admin-user"
# session = boto3.Session(profile_name="rag-admin-user")

# # Accessing STS to assume the role with the right permissions"
# sts = session.client("sts")
# response = sts.assume_role(
#     RoleArn="arn:aws:iam::169189304433:role/RedcoatExpressRagAdmin",
#     RoleSessionName="OpenSearchUpload"
# )

# # Extracting credentials from the session
# credentials = response["Credentials"]

# # Passing credentials to AWSAUTH so we can authenticate to OpenSearch
# awsauth = AWS4Auth(
#     credentials["AccessKeyId"],
#     credentials["SecretAccessKey"],
#     REGION,
#     "es",
#     session_token=credentials["SessionToken"]
# )

# # Creating OpenSearch client with A.) my created OpenSearch domain and B.) my AWSAUTH credentials
# client = OpenSearch(
#     hosts=[{"host": DOMAIN, "port": 443}],
#     http_auth=awsauth,
#     use_ssl=True,
#     verify_certs=True,
#     connection_class=RequestsHttpConnection
# )

# # Testing connection:
# print(client.cat.indices(format="json"))

Access over OpenSearch Master User

In [73]:

client = OpenSearch(
    hosts=[{"host": DOMAIN, "port": 443}],
    http_auth=(MASTER_USER, MASTER_PASSWORD),
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection
)

# Test it:
indecies = client.cat.indices(format="json")
for i in indecies[:3]:
    print(i)
    print()


{'health': 'green', 'status': 'open', 'index': 'emails', 'uuid': '1ZqBbC-WSPyDglErajejBw', 'pri': '5', 'rep': '2', 'docs.count': '0', 'docs.deleted': '0', 'store.size': '25.3mb', 'pri.store.size': '8.4mb'}

{'health': 'green', 'status': 'open', 'index': 'otel-v1-apm-service-map-sample', 'uuid': 'InKqxx9zRB-xtgzA46VoUQ', 'pri': '1', 'rep': '2', 'docs.count': '49', 'docs.deleted': '0', 'store.size': '52.9kb', 'pri.store.size': '17.6kb'}

{'health': 'green', 'status': 'open', 'index': '.plugins-ml-model-group', 'uuid': 'GK4aZPJnTUGosjC69CfIyQ', 'pri': '1', 'rep': '2', 'docs.count': '0', 'docs.deleted': '0', 'store.size': '11.2kb', 'pri.store.size': '228b'}



In [68]:
if not client.indices.exists(INDEX_NAME):
    print(f"[INFO] creating index {INDEX_NAME!r}")
    mapping = {
        "settings": {"index": {"knn": True}},
        "mappings": {
            "dynamic": True,
            "properties": {
                "thread_id":   {"type": "keyword"},
                "embedding":   {"type": "knn_vector", "dimension": 1536},
                "type":        {"type": "keyword"},
                "date":        {"type": "date"},
                "subject":     {"type": "text"},
                "chunk_text":  {"type": "text"},
                "summary_text":{"type": "text"},
                "participants":{"type": "keyword"}
            }
        }
    }
    client.indices.create(INDEX_NAME, body=mapping)

[INFO] creating index 'emails'


Uploading Emails to Index

In [ ]:

def actionas_generator():
    for directory in DATA_DIRS:
        for filename in os.listdir(directory):
            if not filename.endswith(".json"):
                continue
            path = os.path.join(directory, filename)
            with open(path, "r", encoding="utf-8") as f:
                doc = json.load(f)
            yield {
                "_index":  INDEX_NAME,
                "_id":     doc.get("thread_id", filename),
                "_source": doc
                }


print(f"[INFO] Documents ready to index")
total = 0
for i in DATA_DIRS:
    total += len(os.listdir(i))

success = 0
for idx, (ok, result) in enumerate(
    helpers.streaming_bulk(
        client,
        actionas_generator(),
        chunk_size=100,           # smaller batches
        request_timeout=60,       # give it a minute per request
        max_retries=2,
        initial_backoff=2,
        max_backoff=10,
        yield_ok=True
        )
    ):
    if ok:
        success += 1
    else:
        # result looks like {"index": {"_id": "...", "error": {...}}}
        print(f"[ERROR] Doc #{idx} failed to index:", result)

    if idx % verbosity == 0:
        print(f"[INFO] {idx}/{total} uploaded -> {round(success/total*100, 0)}%")

print(f"[DONE ] Finished indexing: {success}/{total} succeeded -> {round(success/total*100, 0)}%")

[INFO] Documents ready to index
[INFO] 0/363 uploaded.
[INFO] 20/363 uploaded.
[INFO] 40/363 uploaded.
[INFO] 60/363 uploaded.
[INFO] 80/363 uploaded.
[INFO] 100/363 uploaded.
[INFO] 120/363 uploaded.
[INFO] 140/363 uploaded.
[INFO] 160/363 uploaded.
[INFO] 180/363 uploaded.
[INFO] 200/363 uploaded.
[INFO] 220/363 uploaded.
[INFO] 240/363 uploaded.
[INFO] 260/363 uploaded.
[INFO] 280/363 uploaded.
[INFO] 300/363 uploaded.
[INFO] 320/363 uploaded.
[INFO] 340/363 uploaded.
[INFO] 360/363 uploaded.
[DONE ] Finished indexing: 363/363 succeeded - 100.0%


In [106]:
# Connect to OpenSearch
client = OpenSearch(
    hosts=[{"host": DOMAIN, "port": 443}],
    http_auth=(MASTER_USER, MASTER_PASSWORD),
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection
)


def inspect_index(client):
    
    # Searching the index w/ max 5 results
    resp = client.search(
    index=INDEX_NAME,
    body={                                          # Returns up to 5 documents matching the keyword
            "size": 10,
            "query": { 
                "match_phrase": {
                    "summary_text": "shipping"
                }
            }
        }
    )

    # Print them out one by one ("hit" being when we find a match)
    for hit in resp["hits"]["hits"]:
        src = hit["_source"]
        print(f"— ID: {hit['_id']}, type: {src['type']}")
        # print either the chunk_text or summary_text
        text = src.get("chunk_text") or src.get("summary_text")
        print("  text:", text.replace("\n"," "))
        print("  embedding length:", len(src["embedding"]))
        print()

inspect_index(client)


— ID: 20250507150541.143824d4b@unknown, type: thread
  text: James from Right Freight, a logistics company, introduces their flexible shipping solutions and provides current ocean freight rates from various Chinese ports to London and Southampton, valid from May 12 to May 31, 2025. He invites recipients to contact him for further information or to discuss shipping needs from other Asian countries.
  embedding length: 1536

— ID: 1ee4b8e9-0a17-4090-be9a-db82934a13b9@alibaba.com, type: thread
  text: Mr. Kerwin Li from AAA China Limited provided updated special shipping rates from various Chinese ports to the UK, valid from May 15 to May 30, and requested feedback on the offers. He also included contact information and an option to unsubscribe from future promotional emails.
  embedding length: 1536

— ID: am6pr0602mb3591dc50c95d4e1045682895db97a@am6pr0602mb3591.eurprd06.prod.outlook.com, type: thread
  text: Kiran Parmar from ShearGold Ltd requested a sea freight rate for shipping gym e

In [116]:
def keyword_search(text, field="summary_text", size=5):
    body = {
      "size": size,
      "query": { "match": { field: text }}
    }
    return client.search(index=INDEX_NAME, body=body)["hits"]["hits"]

def knn_search(query_text, size=5):
    llm_client = OpenAI(api_key=SECRET_KEY)

    q_vec = llm_client.embeddings.create(
        model="text-embedding-ada-002", input=query_text
    ).data[0].embedding

    body = {
      "size": size,
      "query": {
        "knn": {
          "embedding": { "vector": q_vec, "k": size }
        }
      }
    }
    return client.search(index=INDEX_NAME, body=body)["hits"]["hits"]


In [132]:
text = "Who is Sarah Hawkins?"
search = keyword_search(text)[0]["_source"]
search = knn_search(text)[0]["_source"]

for i in search:
    if i == "embedding":
        continue
    print(f"{i}: {search[i]}")

type: thread
thread_id: lo0p123mb66690fa6db7e7f062a866919ae8aa@lo0p123mb6669.gbrp123.prod.outlook.com
subject: Pre alert SN203/12 MAY
participants: ['Sarah Hawkins <sarah@redcoat.org.uk>', 'lgwexports <lgwexports@redcoat.org.uk> <lgwexports@redcoat.org.uk>', 'Halimatou Bah <halimatou.bah@redcoat.org.uk>', "'DHL Express' via bjlexports <bjlexports@redcoat.org.uk>"]
first_date: 2025-05-09T11:20:07+00:00
last_date: 2025-05-09T11:20:07+00:00
summary_text: Sarah Hawkins from Redcoat Express Ltd sent an email with an attached pre-alert document, which appears to be an Air Waybill detailing the shipment of goods, including terms of carriage and insurance options. The document specifies that the goods are accepted in good order and condition, subject to the conditions of the contract, and includes information about the shipment's routing, destination, and handling instructions.
